### 1. 패키지 설치

### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  # RecursiveCharacterTextSplitter로 변경

# 텍스트 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=128  # 오버랩 설정
)

# 단일 DOCX 파일 로드 및 분할
file_path = "./dataset2.docx"  # 파일 경로를 이곳에 입력하세요
loader = Docx2txtLoader(file_path)
documents = loader.load_and_split(text_splitter)

# 문서 수 출력
print(f"총 문서 수: {len(documents)}")

# 첫 5개 문서만 출력해 확인
for i, doc in enumerate(documents[:5], 1):
    print(f"Document {i}:")
    print(doc.page_content)
    print("-" * 50)


총 문서 수: 16754
Document 1:
작품명: 작가사진 / 作家寫眞 / Selfportrait

작가: 한기석 / HAN Kisuk

작품 번호: 1

제작 연도: 1960

크기: 41×51

재료: 종이에 젤라틴실버프린트

카테고리: 사진

작품 설명: ‘농(Nong)’이라는 이름으로 미국에서 널리 알려진 한농(韓農) 한기석(1930-2011)은 국내 활동이 그리 많지 않아서 한국 화단에서는 생소한 이름이다. 그가 최초로 한국 화단에 등장한 것은 1971년 11월 신세계 화랑에서 개최한《Nong 展》이후이다. 그는 농(Nong)을 구름 위의 시선(詩仙) 혹은 주선(酒仙)같은 존재로 비유해서 미국에서 자신의 이름으로 쓰고 있다.그의 작품은 전반적으로 자신의 철학적 이미지를 조형화시킨 추상 회화 계통이다. 일종의 형이상학적인 회화 혹은 초현실적인 환상세계라고도 할 수 있는 그의 작품은 양식적인 면에서 주로 구상적인 형태를 취한다.한기석의 <작가사진>(1960)은 본인의 얼굴을 찍은 것으로, 사진 속에서 작가는 자신의 작품을 배경으로 화면의 우측을 주시하고 있다.





작품명: 팔괘호 / 八卦壺 / Palgwae Vase

작가: 한기석 / HAN Kisuk

작품 번호: 2

제작 연도: 1960

크기: 250×127

재료: 캔버스, 종이에 유화 물감

카테고리: 회화 II
--------------------------------------------------
Document 2:
작품명: 팔괘호 / 八卦壺 / Palgwae Vase

작가: 한기석 / HAN Kisuk

작품 번호: 2

제작 연도: 1960

크기: 250×127

재료: 캔버스, 종이에 유화 물감

카테고리: 회화 II

작품 설명: 한농(韓農) 한기석(1930-2011)은 표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는 항아리와 화병을 계속 그렸는데, 항아리에 대한 사람들의 일반적 향수 이미지는 그의 작품에서 친근감을 불어넣어 준다.

In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  # RecursiveCharacterTextSplitter로 변경
from langchain.schema import Document

# 텍스트 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=128  # 오버랩 설정
)

# 단일 DOCX 파일 로드
file_path = "./dataset2.docx"  # 파일 경로를 이곳에 입력하세요
loader = Docx2txtLoader(file_path)
raw_text = loader.load()[0].page_content  # DOCX 파일의 전체 텍스트 가져오기

# 작품명을 기준으로 텍스트 분리
def split_artwork_documents(doc_text):
    # "작품명:"을 기준으로 텍스트를 분할하여 각 작품 정보 추출
    artworks = doc_text.split("\n\n작품명:")  # 작품 구분을 위한 기준 문자열 사용
    documents = []
    
    for artwork in artworks:
        if artwork.strip():  # 빈 문자열 제외
            # "작품명:"이 누락된 경우 앞에 붙여주어 일관성 유지
            doc_content = "작품명:" + artwork if not artwork.startswith("작품명:") else artwork
            # 각 작품을 개별 Document 객체로 추가
            documents.append(Document(metadata={'source': file_path}, page_content=doc_content.strip()))
    
    return documents

# 작품별 Document 생성
documents = split_artwork_documents(raw_text)

# 생성된 작품별 Document에 대해 추가 청크 분할
chunked_documents = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)
    for chunk in chunks:
        chunked_documents.append(Document(metadata=doc.metadata, page_content=chunk))

# 총 분할된 문서 수 출력
print(f"총 문서 수: {len(chunked_documents)}")

# 첫 2개 문서만 출력해 확인
for i, doc in enumerate(chunked_documents[:2], 1):
    print(f"Document {i}:")
    print(doc.page_content)
    print("-" * 50)


총 문서 수: 18006
Document 1:
작품명: 작가사진 / 作家寫眞 / Selfportrait

작가: 한기석 / HAN Kisuk

작품 번호: 1

제작 연도: 1960

크기: 41×51

재료: 종이에 젤라틴실버프린트

카테고리: 사진

작품 설명: ‘농(Nong)’이라는 이름으로 미국에서 널리 알려진 한농(韓農) 한기석(1930-2011)은 국내 활동이 그리 많지 않아서 한국 화단에서는 생소한 이름이다. 그가 최초로 한국 화단에 등장한 것은 1971년 11월 신세계 화랑에서 개최한《Nong 展》이후이다. 그는 농(Nong)을 구름 위의 시선(詩仙) 혹은 주선(酒仙)같은 존재로 비유해서 미국에서 자신의 이름으로 쓰고 있다.그의 작품은 전반적으로 자신의 철학적 이미지를 조형화시킨 추상 회화 계통이다. 일종의 형이상학적인 회화 혹은 초현실적인 환상세계라고도 할 수 있는 그의 작품은 양식적인 면에서 주로 구상적인 형태를 취한다.한기석의 <작가사진>(1960)은 본인의 얼굴을 찍은 것으로, 사진 속에서 작가는 자신의 작품을 배경으로 화면의 우측을 주시하고 있다.
--------------------------------------------------
Document 2:
작품명: 팔괘호 / 八卦壺 / Palgwae Vase

작가: 한기석 / HAN Kisuk

작품 번호: 2

제작 연도: 1960

크기: 250×127

재료: 캔버스, 종이에 유화 물감

카테고리: 회화 II

작품 설명: 한농(韓農) 한기석(1930-2011)은 표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는 항아리와 화병을 계속 그렸는데, 항아리에 대한 사람들의 일반적 향수 이미지는 그의 작품에서 친근감을 불어넣어 준다. 또한 완벽한 균형을 이루면서 노련하게 표현되어 다양한 색의 조화를 지닌 도자기의 미를 느낄 수 있다.
--------------------------------------------------


In [5]:
documents[254]

Document(metadata={'source': './dataset2.docx'}, page_content='작품명: 산 / 山 / Mountain\n\n작가: 유영국 / YOO Youngkuk\n\n작품 번호: 255\n\n제작 연도: 1962\n\n크기: 173×227.5\n\n재료: 캔버스에 유화 물감\n\n카테고리: 회화 II\n\n작품 설명: 유영국(1916-2002)의 <산>(1962)에서는 구체적 이미지의 재현이 사라지고 기하학적 본질에 접근하는 극도의 단순성이 구현되고 있다. 화면은 전통적인 원근법을 사용하기 보다 짙은 청색과 밝은 황색 등으로 채색되어 있다. 또한 작품에는 짙고 옅은 단색조의 색채 흔적이 크게 자리잡고 있으며, 그것만이 유일하게 표현 대상의 공간성을 알려주고 있다. 이러한 공간적 표현은 공허와 충만을 표상하는 것이며, 한편 색채는 자연의 동적인 성격과 그 변화를 암시하고 있다.')

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

# 올바른 Hugging Face 모델을 사용한 임베딩 생성
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')

# 확인
print("HuggingFaceEmbeddings initialized successfully!")


/tmp/ipykernel_22088/746791366.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')
/home/chae/ch/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


HuggingFaceEmbeddings initialized successfully!


In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# 3. Chroma 데이터베이스 초기화 및 문서 추가
collection_name = 'chroma_art'

database = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    collection_name=collection_name,
    persist_directory='./chroma_1104'
)

print("Chroma database initialized successfully!")

Chroma database initialized successfully!


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [10]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = 'LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 7/7 [00:27<00:00,  3.86s/it]


In [11]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.7,      
    top_k=50,             
    repetition_penalty=1.03
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipykernel_22088/1438945161.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [22]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|> 
당신은 친절한 한국어 예술작품 챗봇입니다. 
오직 문서(context)에 있는 정보만 사용해 사실대로 질문에 답하세요. 
모든 답변은 반드시 한국어(Korean)로 답하세요.
다음과 같은 마크다운 구조로 답변하세요.

- 작품명: 
- 작품 번호: 
- 작가: 
- 제작 연도: 
- 크기:
- 재료:
- 카테고리: 
- 작품 설명: 


<|user|>
{context}

Question: {question}
'''

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [23]:
retriever = database.as_retriever(search_kwargs={"k": 5})

In [24]:
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers.string import StrOutputParser
from langchain.prompts import ChatPromptTemplate

# 체인 구성 수정
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [25]:
query = "김기승의 '낙양성동도이화'에 대해 설명해주세요"

In [26]:
# 마지막에 원하는 형식으로 출력 필터링
response = chain.invoke(query)
print(response)

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Human: 
<|system|> 
당신은 친절한 한국어 예술작품 챗봇입니다. 
오직 문서(context)에 있는 정보만 사용해 사실대로 질문에 답하세요. 
모든 답변은 반드시 한국어(Korean)로 답하세요.
다음과 같은 마크다운 구조로 답변하세요.

- 작품명: 
- 작품 번호: 
- 작가: 
- 제작 연도: 
- 크기:
- 재료:
- 카테고리: 
- 작품 설명: 


<|user|>
[Document(metadata={'source': './dataset2.docx'}, page_content='작품명: 낙양성동도이화(6곡병) / 洛陽城東桃李花(六曲屛) / \n\n작가: 김기승 / KIM Kiseung\n\n작품 번호: 1364\n\n제작 연도: 1963\n\n크기: 128×31.5×(6)\n\n재료: 종이에 먹; 6폭 병풍\n\n카테고리: 서예\n\n작품 설명: 원곡(原谷) 김기승(1909-2000)은 한국 현대 서예사의 대표적인 작가이다. 1946년 소전(素筌) 손재형(孫在馨) 문하에 들어가 본격적인 서예공부를 시작하였고, 《제1-4회 대한민국미술전람회》(1949, 1953-1955)까지 잇달아 서예부 특선을 차지하여 문교부장관상을 수상하였다. 1955년에는 대성서예원(大成書藝院)을 설립하였고, 1978년에는 원곡서예상(原谷書藝賞)을 제정하기도 하였다.김기승은 《제10회 대한민국미술전람회》(1961)의 취지문에서 "한국적 향기와 한국인의 체취를 풍기는 작품을 제작하기 위하여 온몸을 혹사하면서까지 많은 노력을 기울였으며, 서예의 경지를 어느 단계에 끌어올리려고 정성을 다하였다."라고 언급한 바 있다. 한국적 정취를 효과적으로 드러내면서도 특정 형식이나 글자 형태에 제한되지 않고 새롭고 율동적인 필세와 개성적인 감각을 추구하는 김기승의 작품들은 작가의 이러한 취지를 잘 드러낸다.또한, 김기승은 원곡체(原谷體)를 만들어내고 묵영(墨映)을 창안하는 등, 서예계의 원로임에도 불구하고 새로움을 추구하는 데 게을리 하지 않은 작가이다. 작가 자신이 전위적이라고 말

In [ ]:
results = database.similarity_search(query, k=5)  # 상위 3개의 결과를 가져옴

# 결과 확인
for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(result["description"] if "description" in result else result)
    print("-" * 50)

In [22]:
# 검색 수행: 유사도 점수와 함께 반환
docs_and_scores = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 검색된 문서 수 출력
print(f"검색된 문서 수: {len(docs_and_scores)}")

# 각 문서의 파일명, 전체 내용, 유사도 점수 출력
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n문서 {i}:")
    print(f"  파일명: {doc.metadata.get('source', 'N/A')}")
    print(f"  유사도 점수: {score:.4f}")
    print(f"  전체 내용: {doc.page_content}")


검색된 문서 수: 5

문서 1:
  파일명: ./dataset2.docx
  유사도 점수: 0.2597
  전체 내용: 작가: 김기승 / KIM Kiseung

작품 번호: 1364

작품명: 낙양성동도이화(6곡병) / 洛陽城東桃李花(六曲屛) / 

제작 연도: 1963

크기: 128×31.5×(6)

재료: 종이에 먹; 6폭 병풍

카테고리: 서예

작품 설명: 원곡(原谷) 김기승(1909-2000)은 한국 현대 서예사의 대표적인 작가이다. 1946년 소전(素筌) 손재형(孫在馨) 문하에 들어가 본격적인 서예공부를 시작하였고, 《제1-4회 대한민국미술전람회》(1949, 1953-1955)까지 잇달아 서예부 특선을 차지하여 문교부장관상을 수상하였다. 1955년에는 대성서예원(大成書藝院)을 설립하였고, 1978년에는 원곡서예상(原谷書藝賞)을 제정하기도 하였다.김기승은 《제10회 대한민국미술전람회》(1961)의 취지문에서 "한국적 향기와 한국인의 체취를 풍기는 작품을 제작하기 위하여 온몸을 혹사하면서까지 많은 노력을 기울였으며, 서예의 경지를 어느 단계에 끌어올리려고 정성을 다하였다."라고 언급한 바 있다. 한국적 정취를 효과적으로 드러내면서도 특정 형식이나 글자 형태에 제한되지 않고 새롭고 율동적인 필세와 개성적인 감각을 추구하는 김기승의 작품들은 작가의 이러한 취지를 잘 드러낸다.또한, 김기승은 원곡체(原谷體)를 만들어내고 묵영(墨映)을 창안하는 등, 서예계의 원로임에도 불구하고 새로움을 추구하는 데 게을리 하지 않은 작가이다. 작가 자신이 전위적이라고 말하는 '묵영'이란 청묵(靑墨)의 번짐을 사용하거나 먹물의 농담을 이용하여 시각효과를 부각시킨 회화적 서예이다. 일부에서 묵영을 '전통을 무시한 예술'이라고 몰아붙이자 "전통을 지키기 위해서는 다각적인 실험작업을 통해 새로운 조형언어를 만들어내야 한다."고 맞설 정도로 원곡은 새로움을 추구하였다.<낙양성동도이화(6곡병)>은 1963년에 제작된 6곡의 병풍으로 김기승이 손과정의 『서보(書譜)』의 필의로 쓴 초서

In [34]:
import re

def extract_answer(text):
    # 정규 표현식을 사용하여 'Answer:' 이후의 모든 텍스트 추출
    match = re.search(r"답변:\s*(.*)", text, re.DOTALL)
    if match:
        answer = match.group(1).strip()  # 불필요한 공백 제거
    else:
        answer = "답변을 찾을 수 없습니다."
    
    return answer

In [35]:
answer = extract_answer(response)
print(answer)

이 작품의 크기는 약 6미터입니다. 정확한 치수는 2 미터 (높이) x 3 피트 (가로) 입니다. 또한, 이 작품은 실크 스크린 인쇄 기법과 유화 물감을 사용해서 만들어졌으며, 캔버스 위에 그려졌습니다.


In [36]:
docs_and_scores[0][1]

0.23836731910705566

In [67]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 로드
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chroma 데이터베이스 불러오기 (임베딩 함수 포함)
persist_directory = './chroma_huggingface1028'
collection_name = 'chroma_art'

database = Chroma(
    embedding_function=embeddings,  # 임베딩 함수 설정
    collection_name=collection_name,
    persist_directory=persist_directory
)

# 모든 문서 검색을 위한 retriever 생성
retriever = database.as_retriever(search_kwargs={"k": 1000})

# 문서 검색 수행 (빈 쿼리 사용)
all_docs = retriever.get_relevant_documents(" ")

# 파일 소스 목록 출력
file_sources = set(doc.metadata['source'] for doc in all_docs)  # 중복 제거

print("데이터베이스에 저장된 파일 목록:")
for source in file_sources:
    print(source)


데이터베이스에 저장된 파일 목록:
./dataset_part_1.docx
./dataset_part_4.docx
./dataset_part_5.docx
./dataset_part_3.docx
./dataset_part_2.docx


In [39]:
from transformers import pipeline
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser

# 프롬프트 템플릿 설정
template = '''
<|system|> 
당신은 친절한 한국어 예술작품 챗봇입니다. 
오직 문서(context)에 있는 정보만 사용해 사실대로 질문에 답하세요. 
모든 답변은 반드시 한국어(Korean)로 답하세요.

<|user|>
{context}

Question: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# 하이퍼파라미터 값 범위 설정
temperature_values = [0.7, 0.9, 1.1]
top_k_values = [20, 40, 60]
repetition_penalty_values = [1.02, 1.08, 1.2]
max_new_tokens_values = [512, 1024]

# 결과 저장 리스트 초기화
results = []

# 하이퍼파라미터 조합 반복
for temperature in temperature_values:
    for top_k in top_k_values:
        for repetition_penalty in repetition_penalty_values:
            for max_new_tokens in max_new_tokens_values:
                # 파이프라인 생성
                pipe = pipeline(
                    "text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens=max_new_tokens,
                    do_sample=True,
                    temperature=temperature,
                    top_k=top_k,
                    repetition_penalty=repetition_penalty
                )
                
                # LangChain의 HuggingFacePipeline 사용
                llm = HuggingFacePipeline(pipeline=pipe)
                
                # 체인 구성
                chain = (
                    RunnableMap({
                        "context": retriever,
                        "question": RunnablePassthrough()
                    })
                    | prompt
                    | llm
                    | StrOutputParser()
                )
                
                query = "김기승의 낙양성동도이화에 대해 마크다운으로 알려주세요."
                
                # 체인 실행
                generated_text = chain.invoke(query)

                # 결과 저장
                results.append({
                    "temperature": temperature,
                    "top_k": top_k,
                    "repetition_penalty": repetition_penalty,
                    "max_new_tokens": max_new_tokens,
                    "generated_text": generated_text
                })

# 결과 출력
for result in results:
    print(f"Temperature: {result['temperature']}, Top_k: {result['top_k']}, "
          f"Repetition Penalty: {result['repetition_penalty']}, Max New Tokens: {result['max_new_tokens']}")
    print("Generated Text:", result["generated_text"])
    print("-" * 50)


Temperature: 0.7, Top_k: 20, Repetition Penalty: 1.02, Max New Tokens: 512
Generated Text: Human: 
<|system|> 
당신은 친절한 한국어 예술작품 챗봇입니다. 
오직 문서(context)에 있는 정보만 사용해 사실대로 질문에 답하세요. 
모든 답변은 반드시 한국어(Korean)로 답하세요.

<|user|>
[Document(metadata={'source': './dataset2.docx'}, page_content='작가: 김기승 / KIM Kiseung\n\n작품 번호: 1364\n\n작품명: 낙양성동도이화(6곡병) / 洛陽城東桃李花(六曲屛) / \n\n제작 연도: 1963\n\n크기: 128×31.5×(6)\n\n재료: 종이에 먹; 6폭 병풍\n\n카테고리: 서예'), Document(metadata={'source': './dataset2.docx'}, page_content='작가: 김기승 / KIM Kiseung\n\n작품 번호: 1222\n\n작품명: 동방기백묵영 / 東方旣白墨映 / \n\n제작 연도: 1962\n\n크기: 126×65\n\n재료: 종이에 먹\n\n카테고리: 서예'), Document(metadata={'source': './dataset2.docx'}, page_content='작가: 김기승 / KIM Kiseung\n\n작품 번호: 1252\n\n작품명: 시백악천 / 詩白樂天 / \n\n제작 연도: N/A\n\n크기: 67×128\n\n재료: 종이에 먹\n\n카테고리: 서예'), Document(metadata={'source': './dataset2.docx'}, page_content='작가: 김기승 / KIM Kiseung\n\n작품 번호: 1384\n\n작품명: 천주애정의 / 天主愛正義 / \n\n제작 연도: N/A\n\n크기: 132×33\n\n재료: 종이에 먹\n\n카테고리: 서예'), Document(metadata={'source'

In [40]:
from transformers import pipeline
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser

# 프롬프트 템플릿 설정
template = '''
<|system|> 
당신은 친절한 한국어 예술작품 챗봇입니다. 
문서(context)에 있는 정보만 사용해 질문에 답하세요. 
모든 답변은 반드시 한국어(Korean)로 다음의 구조로 대답하세요.

---

## 작품명
- **작가명**: {{작가명}}
- **작품번호**: {{작품번호}}
- **제작 연도**: {{제작연도}}
- **크기**: {{크기}}
- **재료**: {{재료}}
- **카테고리**: {{카테고리}}
- **작품설명**: {{작품설명}}

---

<|user|>
{context}

Question: {question}
'''
prompt = ChatPromptTemplate.from_template(template)

# 하이퍼파라미터 값 범위 설정
temperature_values = [0.7, 0.9, 1.1]
top_k_values = [20, 40, 60]
repetition_penalty_values = [1.02, 1.08, 1.2]
max_new_tokens_values = [512, 1024]

# 결과 저장 리스트 초기화
results = []

# 하이퍼파라미터 조합 반복
for temperature in temperature_values:
    for top_k in top_k_values:
        for repetition_penalty in repetition_penalty_values:
            for max_new_tokens in max_new_tokens_values:
                # 파이프라인 생성
                pipe = pipeline(
                    "text-generation",
                    model=model,
                    tokenizer=tokenizer,
                    max_new_tokens=max_new_tokens,
                    do_sample=True,
                    temperature=temperature,
                    top_k=top_k,
                    repetition_penalty=repetition_penalty
                )
                
                # LangChain의 HuggingFacePipeline 사용
                llm = HuggingFacePipeline(pipeline=pipe)
                
                # 체인 구성
                chain = (
                    RunnableMap({
                        "context": retriever,
                        "question": RunnablePassthrough()
                    })
                    | prompt
                    | llm
                    | StrOutputParser()
                )
                
                query = "김기승의 낙양성동도이화에 대해 마크다운으로 알려주세요."
                
                # 체인 실행
                generated_text = chain.invoke(query)

                # 결과 저장
                results.append({
                    "temperature": temperature,
                    "top_k": top_k,
                    "repetition_penalty": repetition_penalty,
                    "max_new_tokens": max_new_tokens,
                    "generated_text": generated_text
                })

# 결과 출력
for result in results:
    print(f"Temperature: {result['temperature']}, Top_k: {result['top_k']}, "
          f"Repetition Penalty: {result['repetition_penalty']}, Max New Tokens: {result['max_new_tokens']}")
    print("Generated Text:", result["generated_text"])
    print("-" * 50)


Temperature: 0.7, Top_k: 20, Repetition Penalty: 1.02, Max New Tokens: 512
Generated Text: Human: 
<|system|> 
당신은 친절한 한국어 예술작품 챗봇입니다. 
문서(context)에 있는 정보만 사용해 질문에 답하세요. 
모든 답변은 반드시 한국어(Korean)로 다음의 구조로 대답하세요.

---

## 작품명
- **작가명**: {작가명}
- **작품번호**: {작품번호}
- **제작 연도**: {제작연도}
- **크기**: {크기}
- **재료**: {재료}
- **카테고리**: {카테고리}
- **작품설명**: {작품설명}

---

<|user|>
[Document(metadata={'source': './dataset2.docx'}, page_content='작가: 김기승 / KIM Kiseung\n\n작품 번호: 1364\n\n작품명: 낙양성동도이화(6곡병) / 洛陽城東桃李花(六曲屛) / \n\n제작 연도: 1963\n\n크기: 128×31.5×(6)\n\n재료: 종이에 먹; 6폭 병풍\n\n카테고리: 서예'), Document(metadata={'source': './dataset2.docx'}, page_content='작가: 김기승 / KIM Kiseung\n\n작품 번호: 1222\n\n작품명: 동방기백묵영 / 東方旣白墨映 / \n\n제작 연도: 1962\n\n크기: 126×65\n\n재료: 종이에 먹\n\n카테고리: 서예'), Document(metadata={'source': './dataset2.docx'}, page_content='작가: 김기승 / KIM Kiseung\n\n작품 번호: 1252\n\n작품명: 시백악천 / 詩白樂天 / \n\n제작 연도: N/A\n\n크기: 67×128\n\n재료: 종이에 먹\n\n카테고리: 서예'), Document(metadata={'source': './dataset2.docx'}, page_content='작가